In [73]:
import requests
from bs4 import BeautifulSoup
import re
import time

In [3]:
from selenium import webdriver
import time

In [4]:
import pandas as pd
import numpy as np

In [101]:
from datetime import datetime
from datetime import datetime, timezone
import pytz

In [42]:
def slugify(s):
    """
    Simplifies ugly strings into something URL-friendly.
    >>> print slugify("[Some] _ Article's Title--")
    some-articles-title
    """

    # "[Some] _ Article's Title--"
    # "[some] _ article's title--"
    s = s.lower()

    # "[some] _ article's_title--"
    # "[some]___article's_title__"
    for c in [' ', '-', '.', '/']:
        s = s.replace(c, '_')

    # "[some]___article's_title__"
    # "some___articles_title__"
    s = re.sub('\W', '', s)

    # "some___articles_title__"
    # "some   articles title  "
    s = s.replace('_', ' ')

    # "some   articles title  "
    # "some articles title "
    s = re.sub('\s+', ' ', s)

    # "some articles title "
    # "some articles title"
    s = s.strip()

    # "some articles title"
    # "some-articles-title"
    s = s.replace(' ', '-')

    return s

In [112]:
def is_today(article_date):
    intz = pytz.timezone('Asia/Kolkata')
    now_dt = datetime.now(intz)
    article_dt = datetime.strptime(article_date, '%d %b %Y, %H:%M hrs IST')
    return now_dt.day == article_dt.day

In [47]:
delhi_toi = "https://timesofindia.indiatimes.com/city/delhi"
delhi_toi_base_url = "https://timesofindia.indiatimes.com"
ltst_th = "https://www.thehindu.com/news/"

In [113]:
def fetch_toi_daily_news(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    name_box = soup.find(attrs={'id': 'c_articlelist_stories_2'})
    # text_a = [s.get('href').strip() + '|' + s.getText().strip() for s in name_box.findAll('a', href=True)]
    lis = [s for s in name_box.findAll('li')]
    for li in lis:
        data = []
        found = False
        for s in li.findAll('a', href=True):
            article_link = delhi_toi_base_url + s.get('href').strip() 
            article_title = s.getText().strip()
            article_id = slugify(article_title)
            data = [article_id, article_title, article_link]
            found = True
            break
        if found:
            for s in li.findAll(attrs={'class': 'strlastupd'}):
                article_date = s.getText().strip()
                if not is_today(article_date):
                    return ""
                data.append(article_date)
                news_data.append(data)
    ul_page = name_box.find(attrs={'class': 'pagination'})
    lis = [s for s in ul_page.findAll('li')]
    pages = []
    next_link = None
    found_curr = False
    for li in lis:
        if found_curr:
            for s in li.findAll('a', href=True):
                next_link = delhi_toi_base_url + s.get('href').strip()
                break
            break
        curr = li.get('class')
        if curr:
            found_curr = True
    return next_link

In [114]:
news_data = []
url = delhi_toi
continue_now = True
count = 0
while continue_now and count < 15:
    time.sleep(5)
    url = fetch_toi_daily_news(url)
    print(url)
    print(news_data[-1])
    if not url:
        continue_now = False
    count += 1


['delhi-soon-take-metro-to-manesar-rewari', "Delhi: Soon, take 'Metro' to Manesar, Rewari", 'https://timesofindia.indiatimes.com/city/delhi/delhi-soon-take-metro-to-manesar-rewari/articleshow/67263858.cms', '27 Dec 2018, 06:43 hrs IST']


In [116]:
len(news_data)

29

In [118]:
news_df = pd.DataFrame(news_data)
news_df.columns = ["article_id", "article_title", "article_url", "article_time"]

In [119]:
news_df.head()

,article_id,article_title,article_url,article_time
0,delhi-lawyer-plays-cop-nabs-snatcher-in-12-hrs,"Delhi lawyer plays cop, nabs snatcher in 12 hrs",https://timesofindia.indiatimes.com/city/delhi...,"27 Dec 2018, 01:26 hrs IST"
1,kiosks-at-delhi-metro-stations-to-help-you-boo...,Kiosks at Delhi metro stations to help you boo...,https://timesofindia.indiatimes.com/city/delhi...,"27 Dec 2018, 12:30 hrs IST"
2,200-schools-under-scanner-for-affiliation-with...,200 schools under scanner for affiliation with...,https://timesofindia.indiatimes.com/city/delhi...,"27 Dec 2018, 08:16 hrs IST"
3,plan-to-give-dilli-haat-a-fresh-look,Plan to give Dilli Haat a fresh look,https://timesofindia.indiatimes.com/city/delhi...,"27 Dec 2018, 06:56 hrs IST"
4,delhi-hospitals-sign-up-for-central-scheme,Delhi hospitals sign up for central scheme,https://timesofindia.indiatimes.com/city/delhi...,"27 Dec 2018, 02:38 hrs IST"


In [122]:
file_name = "sample.csv"
news_df.to_csv(file_name, sep='|', encoding='utf-8')
df = pd.read_csv(file_name, delimiter='|')
df.head()

,Unnamed: 0,article_id,article_title,article_url,article_time
0,0,delhi-lawyer-plays-cop-nabs-snatcher-in-12-hrs,"Delhi lawyer plays cop, nabs snatcher in 12 hrs",https://timesofindia.indiatimes.com/city/delhi...,"27 Dec 2018, 01:26 hrs IST"
1,1,kiosks-at-delhi-metro-stations-to-help-you-boo...,Kiosks at Delhi metro stations to help you boo...,https://timesofindia.indiatimes.com/city/delhi...,"27 Dec 2018, 12:30 hrs IST"
2,2,200-schools-under-scanner-for-affiliation-with...,200 schools under scanner for affiliation with...,https://timesofindia.indiatimes.com/city/delhi...,"27 Dec 2018, 08:16 hrs IST"
3,3,plan-to-give-dilli-haat-a-fresh-look,Plan to give Dilli Haat a fresh look,https://timesofindia.indiatimes.com/city/delhi...,"27 Dec 2018, 06:56 hrs IST"
4,4,delhi-hospitals-sign-up-for-central-scheme,Delhi hospitals sign up for central scheme,https://timesofindia.indiatimes.com/city/delhi...,"27 Dec 2018, 02:38 hrs IST"
